In [11]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import os
from statistics import mean

fp = "G:\\My Drive\\Darby Work\\Ytsma and Dyar 2021 (LOD paper)\\"

### Braga method

In [15]:
# define blank/noise regions in ChemLIBS and ChemCam spectra
blank_braga = pd.read_csv(fp + "figures\\braga_noise_regions.csv") 

# import all spectra
dp = fp + 'datasets//'
cc_mars = pd.read_csv(dp+'CC_norm3_spectra.csv')
cc_mars_unnorm = pd.read_csv(dp+'CC_unnorm_spectra.csv')
cl_mars = pd.read_csv(dp+'CL_Mars_norm3_spectra_matched.csv')
cl_earth = pd.read_csv(dp+'CL_Earth_norm3_spectra_matched.csv')
cl_vac = pd.read_csv(dp+'CL_Vacuum_norm3_spectra_matched.csv')
cl_mars_unnorm = pd.read_csv(dp+'CL_Mars_unnorm_spectra_matched.csv')
cl_earth_unnorm = pd.read_csv(dp+'CL_Earth_unnorm_spectra_matched.csv')
cl_vac_unnorm = pd.read_csv(dp+'CL_Vacuum_unnorm_spectra_matched.csv')

In [16]:
# filter regions
df_list = [cc_mars, cc_mars_unnorm, cl_earth, cl_mars, cl_vac, cl_earth_unnorm, cl_mars_unnorm, cl_vac_unnorm]

for df in tqdm(df_list):
    for row in tqdm(df.index, leave=False):
        nm = df['wave'][row]
        # remove rows below first region
        if nm < blank_braga['start'][0]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows after last region
        elif nm > blank_braga['stop'][len(blank_braga)-1]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows between the regions
        for region in range(len(blank_braga)-1):
            if (nm > blank_braga['stop'][region]) & (nm < blank_braga['start'][region+1]):
                df.drop(row, axis = 'index', inplace=True)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

In [29]:
# redefine with updated dfs
df_list = [cc_mars, cc_mars_unnorm, cl_earth, cl_mars, cl_vac, cl_earth_unnorm, cl_mars_unnorm, cl_vac_unnorm]
inst_list = ['CC', 'CC', 'CL', 'CL', 'CL', 'CL', 'CL', 'CL']
atm_list = ["Mars", "Mars", "Earth", "Mars", "Vacuum", "Earth", "Mars", "Vacuum"]
norm_list = ['norm3', 'unnorm', 'norm3', 'norm3', 'norm3', 'unnorm', 'unnorm', 'unnorm']

In [30]:
# summarise stdev across each row, take average
sens_list = []
for df in df_list:
    sens = round(df.set_index('wave').std(axis=1).mean(),9)
    sens_list.append(sens)

# make dataframe
braga_sensitivities = pd.DataFrame({
    "instrument" : inst_list,
    "normalization" : norm_list,
    "atmosphere" : atm_list,
    "sensitivity" : sens_list
})

### Metals method
#### ChemLIBS

In [19]:
sp = dp + 'sensitivity//'
earth_meta = pd.read_csv(sp+'metals_Earth_meta.csv')
mars_meta = pd.read_csv(sp+'metals_Mars_meta.csv')
vac_meta = pd.read_csv(sp+'metals_Vacuum_meta.csv')

earth_unnorm = pd.read_csv(sp+'metals_Earth_unnorm.csv')
mars_unnorm = pd.read_csv(sp+'metals_Mars_unnorm.csv')
vac_unnorm = pd.read_csv(sp+'metals_Vacuum_unnorm.csv')
earth_norm3 = pd.read_csv(sp+'metals_Earth_norm3.csv')
mars_norm3 = pd.read_csv(sp+'metals_Mars_norm3.csv')
vac_norm3 = pd.read_csv(sp+'metals_Vacuum_norm3.csv')

In [20]:
df_list = [earth_unnorm, mars_unnorm, vac_unnorm, earth_norm3, mars_norm3, vac_norm3]
atm_list = ['Earth', 'Mars', 'Vacuum', 'Earth', 'Mars', 'Vacuum']
norm_list = ['unnorm', 'unnorm', 'unnorm', 'norm3', 'norm3', 'norm3']
elems = ['AL', 'CR', 'CU', 'FE', 'SN', 'V']

In [23]:
count = 0
sens_list = []

for df in df_list:
    
    atm = atm_list[count]
    count+=1
    
    if atm == 'Earth':
        meta = earth_meta
    elif atm == 'Mars':
        meta = mars_meta
    elif atm == 'Vacuum':
        meta = vac_meta
    
    sens = []
    
    for element in elems:
        
        spec_names = list(meta[meta['Sample Name'] == str(element+'METAL')]['pkey'])
        spectra = df[spec_names]
    
        # calculate sensitivity
        sensitivity = round(spectra.std(axis=1).mean(),9)
        sens.append(sensitivity)
        
    # average results from all elements
    sens = mean(sens)
    sens_list.append(sens)

# make dataframe of all results
mhc_metal_sens = pd.DataFrame({
    "instrument" : ['CL'] * 6,
    "normalization" : norm_list,
    "atmosphere" : atm_list,
    "sensitivity" : sens_list
})

#### LANL

In [24]:
norm_25 = pd.read_csv(sp+'25_tio2_spectra_norm3.csv')
unnorm_25 = pd.read_csv(sp+'25_tio2_spectra_unnorm.csv')
norm_50 = pd.read_csv(sp+'50_tio2_spectra_norm3.csv')
unnorm_50 = pd.read_csv(sp+'50_tio2_spectra_unnorm.csv')
norm_75 = pd.read_csv(sp+'75_tio2_spectra_norm3.csv')
unnorm_75 = pd.read_csv(sp+'75_tio2_spectra_unnorm.csv')

In [25]:
norm_sens = []
for df in [norm_25, norm_50, norm_75]:
    sens = round(df.set_index('wave').std(axis=1).mean(),9)
    norm_sens.append(sens)
norm_sens = mean(norm_sens)

unnorm_sens = []
for df in [unnorm_25, unnorm_50, unnorm_75]:
    sens = round(df.set_index('wave').std(axis=1).mean(),9)
    unnorm_sens.append(sens)
unnorm_sens = mean(unnorm_sens)

lanl_metal_sens = pd.DataFrame({
    "instrument" : ['CC']*2,
    "normalization" : ['norm3', 'unnorm'],
    "atmosphere" : ['Mars']*2,
    "sensitivity" : [norm_sens, unnorm_sens]
})

Combine results

In [31]:
# merge
metals_sensitivities = pd.concat([lanl_metal_sens, mhc_metal_sens])

metals_sensitivities['method'] = "metals"
braga_sensitivities['method'] = "braga"

sensitivities = pd.concat([metals_sensitivities, braga_sensitivities]).reset_index(drop=True)

In [32]:
sensitivities.to_csv(fp+'instrument_sensitivities.csv', index=False)